<a href="https://colab.research.google.com/github/PhamLeQuangNhat/CS114.K21.KHTN/blob/master/Sarcasm/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Danh sách nhóm**: 
+ Phạm Xuân Trí  - 18521530 
+ Phạm Lê Quang Nhật - 18520120

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My Drive/ML/Demo 

/content/drive/My Drive/ML/Demo


#**1. Mô tả lại bài toán và cách thu thập dataset**
Bài toán **Sacrasm Detection** đây là bài toán phát hiện có tính châm biến trong một câu văn.
* Input: Một câu văn 
* Ouput: Câu văn được phân lớp là 1 nếu là câu châm biến, ngược lại nó được phân lớp là 0.

Cụ thể trong bài toán này chúng em dựa vào những headline của các bài báo điện tử trên trang **"The Onion"** và **"HuffPost"** để xây dựng mô hình dự đoán. Trang "The Onion" là trang chỉ chứa các headline là câu châm biến. Ngược lại với "The Onion", "HuffPost" là trang chỉ chứa các headline không phải là câu châm biến. 

**Cách thu thập dữ liệu**
* Dữ liệu huấn luyện: Dữ liệu được download trên trang [kaggle](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection) được lưu trong file.json. Mỗi dòng dữ liệu bao gồm: 
     * **article**: Link dẫn tới bài báo tương ứng
     * **headline**: Tiêu đề của bài báo cần phân loại
     * **is sacrasm**: Tiêu đề có phải là châm biến hay không. Nếu có mang giá trị 1, ngược lại là 0
     
Có tất cả **28619** dòng dữ liệu, với **x** dòng là châm biến, còn lại **y** dòng là không châm biến bộ dữ liệu này được trích ra 75% để huấn luyện.  
* Dữ liệu kiểm thử
    +  Lấy 25% từ bộ dữ liệu download đã nêu trên
* Dữ liệu test
    +  Sẽ tiến hành crawl trên 2 trang báo điện tử ở trên. Tổng cộng có 2000 headline mới. Mỗi trang 1000 headline 


#**2.Thu thập dữ liệu mới**

#### **Dữ liệu mới từ Onion**

In [ ]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import csv

'''  
 brief_ids và photo_ids là các id tưng ứng của  dẫn tới link các bài báo khác nhau

'''
brief_ids = [1592311200653, 1591726080182, 1591127280478, 1590584520209, 
             1589894280339, 1589294340828, 1588711020164, 1588080060229, 
             1587147720532, 1586546460312, 1586184540217, 1585581600531,
             1582920840434, 1582639200695, 1582128720515, 1581532200479,
             1581099360048, 1580744340055, 1580142900840, 1579637820343,
             1579096620327, 1578410280620, 1576700820848, 1576184280994,
             1575574260415, 1575036000486, 1574361720213, 1573762560848,
             1573477200438, 1572900480129, 1572348600499, 1571938620032,
             1571419440061]

photo_ids = [1590159060816, 1588708260098, 1587653040878, 1585925400900,
             1584127800188, 1582815000473, 1582061760395, 1580835900882,
             1579528800910, 1576610760393, 1574702400578, 1573153500106,
             1571849700244, 1570194000584, 1569247200599, 1567182180912,
             1565892720386, 1564614900603, 1563235200067, 1560963960321,
             1559133180370, 1557413700296, 1556108520577, 1554495180638,
             1552834800968, 1551123540512, 1550160960071, 1548972300054,
             1547739780965, 1544713860139, 1543513440446, 1541701980758,
             1539720900721]


In [ ]:
def Onion_crawler(tag, ids, num_articles, label=1):
    count = 0
    for id in ids:
      source = requests.get("https://www.theonion.com/c/{}?startTime={}".format(tag,id)).text
      soup = BeautifulSoup(source, 'lxml')
      for article in soup.find_all('article'):
          headline = article.h2.text
          csv_writer.writerow([label, headline])
          count += 1
          if count == num_articles:
              print(f"Crawled {count} headlines from {tag} tag")
              return

In [ ]:
csv_file = open('onion_headlines.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['is_sarcastic', 'headline'])

23

In [ ]:
Onion_crawler("news-in-brief", brief_ids, num_articles=644, label=1)
Onion_crawler("news-in-photos", photo_ids, num_articles=643, label=1)
csv_file.close()

Crawled 644 headlines from news-in-brief tag
Crawled 643 headlines from news-in-photos tag


#### **Dữ liệu mới từ Huffpost**

Ta lấy data từ API của HuffPost.

- Đầu tiên gửi request lấy [500 bài báo đầu tiên](https://www.huffpost.com/api/department/news/cards?page=1&limit=500) từ API. Tại đây ta copy toàn bộ file json và lưu vào file huffpost_part1.json.
-  Tiếp theo ta gửi request lấy [500 bài báo tiếp theo](https://www.huffpost.com/api/department/news/cards?page=2&limit=500) từ API. Tuy nhiên lần này API chỉ trả về 194 bài báo. HuffPost API chỉ cho ta request 694 bài báo gần đây. Ta copy file json và lưu vào file huffpost_part2.json.
    chen hinh here



In [ ]:
import json

In [ ]:
with open('part1.json') as f:
    part1 = json.load(f)

with open('part2.json') as f:
    part2 = json.load(f)

csv_file = open('huffpost_headlines.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['is_sarcastic', 'headline'])

for article in part1["cards"]:
    headline = article["headlines"][0]["text"]
    csv_writer.writerow([0, headline])
    
for article in part2["cards"]:
    headline = article["headlines"][0]["text"]
    csv_writer.writerow([0, headline])

csv_file.close()

#**3. Mô tả cách xử lý dữ liệu, feature engineering trên dataset đã cho.**

####  Cài đặt các thư viện cần thiết

In [3]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

import numpy as np 
import pandas as pd 
import re,string

In [5]:
!ls

final.ipynb		part1.json	       Sarcasm.ipynb
huffpost_headlines.csv	part2.json
onion_headlines.csv	SarcasmDatasetv2.json


#### Tải dữ liệu train và phân chia thành 2 phần: tập validation và tập train

In [11]:
data_train = pd.read_json("SarcasmDatasetv2.json",lines=True)

In [12]:
X = data_train['headline']
Y = data_train['is_sarcastic']

#split validation:25%, train:75%
trainX,valX,trainY,valY = train_test_split(X,Y,test_size=0.25,random_state=42)

#### Tải dữ liệu test

In [13]:
onion = pd.read_csv("onion_headlines.csv")
onion_X = onion['headline']
onion_Y = onion['is_sarcastic']

huffpost = pd.read_csv("huffpost_headlines.csv")
huffpost_X = huffpost['headline']
huffpost_Y = huffpost['is_sarcastic']

#### Tiền xử lí dữ liệu 
- Các headline sẽ được chuyển về lowercase, loại bỏ các ký tự đặc biệt.

In [14]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [15]:
trainX = trainX.apply(clean_text)

In [16]:
valX = valX.apply(clean_text)

In [17]:
onion_X = onion_X.apply(clean_text)
huffpost_X = huffpost_X.apply(clean_text)

#### Feature engineering cho bộ data train,test

- Sử dụng TfidfVectorizer để trích xuất đặc trưng.
- Khai báo TfidfVectorizer từ thư viện sklearn để biểu diễn dữ liệu train dưới dạng vector và tạo một bộ từ điển từ dữ liệu train
- Sử dụng lại từ điển của dữ liệu train để biểu diễn dữ liệu validation, test dưới dạng vector
*Hàm TfidfTransformer để đánh giá lại về giá trị các từ bởi vì có những từ tuy xuất hiện nhiều lần nhưng nó lại thực sự không quan trọng (the, and, is,of,that,...)=> giúp model đạt kết quả chính xác hơn

Giải thích về cách tính Tfidf:

*Tfidf = Tf * idf,với:

-Tf: tần số xuất hiện của một từ trong văn bản đó = Số lần xuất hiện của từ nhất định / Số lần xuất hiện nhiều nhất của một từ bất kỳ

-idf: nghịch đảo tần số của văn bản chứa từ nhất định = log(Tổng số văn bản / số văn bản chứa từ đó+1)

Những từ không quan trọng (the,and,it,is,...) thường xuất hiện nhiều trong các văn bản cho nên theo công thức trên ta thấy số văn bản chứa từ đó càng nhiều thì idf càng thấp => tfidf càng thấp => tầm quan trọng của từ đó thấp


In [18]:
# khởi tạo TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1,2), max_features=40000, min_df=2)

# chuyển words sang vetor
trainX = tf.fit_transform(trainX.values).toarray()
trainVocab = tf.vocabulary_ 
tf = TfidfVectorizer(vocabulary=trainVocab)

valX = tf.fit_transform(valX.values).toarray()

onion_X = tf.fit_transform(onion_X.values).toarray()
huffpost_X = tf.fit_transform(huffpost_X.values).toarray()
testX = np.concatenate((onion_X,huffpost_X),axis=0)
testY = np.concatenate((onion_Y,huffpost_Y),axis=0)
print("[INFO] Used TfidfVectorizer ... ")

[INFO] Used TfidfVectorizer ... 


#**4. Mô tả quá trình chọn model, học và fine tuning**

- Vì đây là bài toán phân lớp nên ta sẽ chọn những thuật toán phân lớp như Suport vector classifications,Multinomial Naive Bayes, Logistic Regression

In [19]:
model1 = LogisticRegression()
model2 = MultinomialNB()
model3 = LinearSVC()

In [20]:
for i in range(3):
    
    if i == 0:

        print("   [INFO] evaluating Logistic Regression...")

        # train and evaluating 
        model1.fit(trainX, trainY)
        predict_val_1 = model1.predict(valX)
        predict_test_1 = model1.predict(testX)

        print(classification_report(valY,predict_val_1))

    if i == 1:
        
        print("   [INFO] evaluating Naive Bayes...")

        # train and evaluating 
        model2.fit(trainX, trainY)
        predict_val_2 = model2.predict(valX)
        predict_test_2 = model2.predict(testX)

        print(classification_report(valY,predict_val_2))

    if i == 2:

        print("   [INFO] evaluating SVM...")

        model3.fit(trainX, trainY)
        predict_val_3 = model3.predict(valX)
        predict_test_3 = model3.predict(testX)

        print(classification_report(valY,predict_val_3))

   [INFO] evaluating Logistic Regression...
              precision    recall  f1-score   support

           0       0.85      0.81      0.83      3730
           1       0.80      0.84      0.82      3425

    accuracy                           0.82      7155
   macro avg       0.82      0.83      0.82      7155
weighted avg       0.83      0.82      0.82      7155

   [INFO] evaluating Naive Bayes...
              precision    recall  f1-score   support

           0       0.82      0.88      0.85      3730
           1       0.86      0.79      0.82      3425

    accuracy                           0.83      7155
   macro avg       0.84      0.83      0.83      7155
weighted avg       0.84      0.83      0.83      7155

   [INFO] evaluating SVM...
              precision    recall  f1-score   support

           0       0.83      0.83      0.83      3730
           1       0.82      0.82      0.82      3425

    accuracy                           0.83      7155
   macro avg       0

#**5. Mô tả cách dùng model đã train để viết một đoạn chương trình ngắn, thực hiện sacarsm detection cho một headline bất kỳ được nhập vào.**

#### Nhập một headline bất kỳ 

In [21]:
headline = input()

Crowds Tear Down Statues At Wisconsin Capitol, Attack State Senator


#### Tiền xử lí dữ liệu 
- Headline sẽ được chuyển về lowercase, loại bỏ các ký tự đặc biệt.

In [22]:
headline = [clean_text(headline)]

#### Mã hoá headline

In [23]:
headline = tf.fit_transform(headline).toarray()

#### Dự đoán

In [24]:
predict = model2.predict(headline)

In [25]:
if predict == 1:
    print("This is sarcastic_from the onion")
else:
    print("This isn't sarcastic_from the huffpost")

This isn't sarcastic_from the huffpost


#**6. Đối chiếu performance của model trên dataset đã cho và trên 2000 headine mới. Nhận xét về bài toán này.**


#### Đánh giá mô hình Multinomial Naive Bayes trên dataset đã cho và trên 2000 headine mới.

In [26]:
print(classification_report(valY,predict_val_2))
print(classification_report(testY,predict_test_2))


              precision    recall  f1-score   support

           0       0.82      0.88      0.85      3730
           1       0.86      0.79      0.82      3425

    accuracy                           0.83      7155
   macro avg       0.84      0.83      0.83      7155
weighted avg       0.84      0.83      0.83      7155

              precision    recall  f1-score   support

           0       0.64      0.83      0.73       694
           1       0.89      0.75      0.82      1287

    accuracy                           0.78      1981
   macro avg       0.77      0.79      0.77      1981
weighted avg       0.81      0.78      0.79      1981



#### Nhận xét



Hiệu suất sụt giảm so với bộ dữ liệu cũ vì có những tin tức mới hay nhiều word mới có giá trị TFIDF lớn liên quan đến bài toán không có trong bộ từ vựng đã xây dựng  => sai lệch => giảm hiệu suất

Nhận xét về nhóm bài toán này: Đây là bài toán khó, không thể đạt độ chính xác cao nếu chỉ sử dụng machine learning vì tin tức mỗi ngày mỗi khác, mỗi đổi mới nếu không cập nhật các tin tức mà chỉ sử dụng các bag of words cũ đã được thu thập từ trước thì sẽ thiếu hụt các từ có giá trị cao(weight).

Giải pháp: nên sử dụng deep learning và thực hiện mã hoá bộ từ vựng theo phương pháp word2vec để giải quyết bài toán này
